In [22]:
!pip install https://github.com/intersystems-community/intersystems-irispython/releases/download/3.8.0/intersystems_iris-3.8.0-py3-none-any.whl

In [23]:
import intersystems_iris.dbapi._DBAPI as dbapi
import time
import os

namespace="USER"
port = 1972
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
connection_string = f"{hostname}:{port}/{namespace}"
username = "demo"
password = "demo"

In [24]:
# Note: Ideally conn and cursor should be used with context manager or with try-execpt-finally 
conn = dbapi.connect(connection_string, username, password)
cursor = conn.cursor()

In [25]:
users_table_name = "Users"
users_table_definition = """(
    user_id INTEGER PRIMARY KEY,
    name VARCHAR(255),
    date_of_birth DATE,
    medical_conditions TEXT,
    profile_picture BLOB
)"""

cursor.execute(f"CREATE TABLE IF NOT EXISTS {users_table_name} {users_table_definition}")

0

In [26]:
memories_table_name = "Memories"
memories_table_definition = """(
    memory_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    title TEXT,
    image BLOB,
    description TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
)"""

cursor.execute(f"CREATE TABLE IF NOT EXISTS {memories_table_name} {memories_table_definition}")

0

In [27]:
cursor.execute("ALTER TABLE Memories ADD COLUMN embedding VECTOR(DOUBLE, 1536)")

DatabaseError: [SQLCODE: <-306>:<Column with this name already exists>]
[Location: <ServerLoop>]
[%msg: <Column of name 'embedding' already exists>]

In [28]:
people_table_name = "People"
people_table_definition = """(
    person_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    name VARCHAR(255),
    relationship TEXT,
    description TEXT,
    picture BLOB,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
)"""

cursor.execute(f"CREATE TABLE IF NOT EXISTS {people_table_name} {people_table_definition}")

0

In [16]:
tableName = "Demo.Diagnoses"
tableDefinition = "(row_id INTEGER, icd9_code VARCHAR(255), short_title VARCHAR(255), long_title VARCHAR(''))"

In [17]:
try:
    cursor.execute(f"DROP TABLE {tableName}")  
except:
    pass
cursor.execute(f"CREATE TABLE {tableName} {tableDefinition}")

0

In [18]:
## batch update
sql = "Insert into Demo.Diagnoses (row_id, icd9_code,short_title,long_title) values (?, ?, ?,?)"
params = [('0', '0', "entered from batch update 1","entered from batch update 1"), ('0', '0','entered from batch update 2',"entered from batch update 2"), ('0', '0',"entered from batch update 3", "entered from batch update 3")]
cursor.executemany(sql, params) 

3

In [30]:
def insert_user(name, date_of_birth, medical_conditions, profile_picture):
    try:
        cursor.execute("""
            INSERT INTO Users (user_id, name, date_of_birth, medical_conditions, profile_picture) 
            VALUES (?, ?, ?, ?, ?)
        """, [1, name, date_of_birth, medical_conditions, profile_picture])        
        
    except Exception as e:
        print(e)

# Insert a user and get their ID
user_id = insert_user('John Doe', '1990-01-01', 'None', 'profile_afadsfasdff')

In [32]:
cursor.execute("DELETE * FROM Users")
cursor.fetchall()

DatabaseError: [SQLCODE: <-1>:<Invalid SQL statement>]
[Location: <Prepare>]
[%msg: < IDENTIFIER expected, * found ^DELETE *>]